In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
from labrotation.two_photon_session import TwoPhotonSession
import labrotation.file_handling as fh
from matplotlib import pyplot as plt
import matplotlib
import os
import h5py
import pims_nd2
import numpy as np
import math

### Set output folder

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

### Set parameters

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 20}

matplotlib.rc('font', **font)

# Open file

In [ ]:
nd2_fpath = fh.open_file("Open nikon nd2 file!")

In [ ]:
nik_data = pims_nd2.ND2_Reader(nd2_fpath)

# Limit to specific pixels
The assumption is that even though the voltage indicator has negative signal (i.e. activity = lower fluorescence level), the baseline is higher where there is expression.  

In [ ]:
PIXEL_PERCENT = 0.1  # 0.1 corresponds to 10%, i.e. take 10% of total pixels for trace extraction

res = nik_data[0].shape
n_pixels = res[0]*res[1]

n_selected_pixels = math.ceil(n_pixels*PIXEL_PERCENT)

pixel_percent_100 = PIXEL_PERCENT*100  # in %, i.e. max is 100

In [ ]:
baseline_img = np.array(nik_data[0:10]).mean(axis=0)

## Get pixels coordinates

In [ ]:
# flatten the baseline image, sort indices by value. Need to flip it from ascending to 
# descending order: then first n elements will be the largest (i.e. brightest pixels)
selected_pixels = np.flip(np.argsort(baseline_img.flatten()))[:n_selected_pixels]
# unravel into the 2d indexing
row_indices, col_indices = np.unravel_index(selected_pixels, baseline_img.shape)

## Check resulting pixels
Yellow/green marks the selected pixels, contrasted with dark blue background

In [ ]:
binary_img = np.zeros(baseline_img.shape)
binary_img[row_indices,col_indices] = 1.0

In [ ]:
fig = plt.figure(figsize=(18,18))
plt.imshow(baseline_img)
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(18,18))
plt.imshow(binary_img)
plt.show()

# Extract mean trace of selected pixels

In [ ]:
# [:, row_indices, col_indices] flattens the array into shape (n_frames, n_selected_pixels)
mean_trace = np.array(nik_data)[:,row_indices, col_indices].mean(axis=1) 

# Compare with whole FOV mean trace

In [ ]:
mean_trace_whole_fov = np.array(nik_data).mean(axis=(1,2))

In [ ]:
mean_trace_whole_fov.shape

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=1,figsize=(18,12), sharex=True, sharey=False)

axs[0].plot(mean_trace)
axs[0].title.set_text(f'Brightest {pixel_percent_100}% pixels mean')

axs[1].plot(mean_trace_whole_fov)
axs[1].title.set_text('Whole FOV mean')

plt.xlim((9500, 30000))
axs[0].set_ylim((40,95))
axs[1].set_ylim((35,60))

axs[0].set_ylabel('fluorescence (a.u.)')
axs[1].set_ylabel('fluorescence (a.u.)')

axs[1].set_xlabel("frame")

plt.show()

# Create summary plot
For reporting results per recording

In [ ]:
save_summary_fig = False


fig, axs = plt.subplots(nrows=4, ncols=1,figsize=(18,24), sharex=False, sharey=False, gridspec_kw={'height_ratios': [3, 3, 1, 1]})

axs[0].plot(mean_trace)
axs[0].title.set_text(f'Brightest {int(pixel_percent_100)}% pixels mean')

axs[1].plot(mean_trace_whole_fov)
axs[1].title.set_text('Whole FOV mean')

axs[0].set_xlim((9500, 30000))
axs[1].set_xlim((9500, 30000))
axs[0].set_ylim((40,95))
axs[1].set_ylim((35,60))

axs[0].set_ylabel('fluorescence (a.u.)')
axs[1].set_ylabel('fluorescence (a.u.)')

axs[1].set_xlabel("frame")


axs[2].imshow(baseline_img)
axs[2].title.set_text("FOV image")


axs[3].title.set_text(f"Brightest {int(pixel_percent_100)}% pixels")
axs[3].imshow(binary_img)
plt.tight_layout()
if save_summary_fig:
    output_fpath = os.path.join(env_dict["DOWNLOADS_FOLDER"], f"voltage_comparison_{fh.get_datetime_for_fname()}.jpg")
    plt.savefig(output_fpath)
    print(f"Saved to {output_fpath}")
plt.show()